In [1]:
%load_ext autoreload
%autoreload 2

import time
import random
import pickle, os
import numpy as np
import cvxpy as cp

from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

from free_flyer.free_flyer import FreeFlyer
from free_flyer.utils import *
from solvers.coco_ff import CoCo_FF
from solvers.meta_ff import Meta_FF

In [5]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()
x_train = train_data[1]
y_train = train_data[3]

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
p_test, x_test, u_test, y_test, c_test, times_test = test_data
test_file.close()

n_test = x_test.shape[0]

In [7]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles_map']

meta_learner = Meta_FF(system, prob, prob_features)

n_features = 4
device_id= 1
meta_learner.construct_strategies(n_features, train_data, device_id=device_id)

In [8]:
fn_saved = 'models/CoCoFF_free_flyer_20210406_0808.pt'
meta_learner.load_network(fn_saved, False)

meta_learner.model_fn

Loading presaved classifier model from models/CoCoFF_free_flyer_20210406_0808.pt


'/home/acauligi/cs_234/project/mlopt-micp/MetaCoCoFF_free_flyer_20210425_1938.pt'

In [ ]:
meta_learner.train(train_data, "runs/free_flyer")
# meta_learner.finetuning(test_data, "runs/finetunning")

In [ ]:
prob_params_list = []
for ii in range(2):
    prob_params = {}
    for k in p_test.keys():
        prob_params[k] = p_test[k][ii]
    prob_params_list.append(prob_params)
meta_learner.forward(prob_params_list)